In [ ]:
!pip install ebooklib tqdm deep_translator IPython urllib3

## **Code Sections**:

1. **Importing Necessary Libraries and Modules:**
   - This section imports the required Python libraries and modules, including those for working with EPUB files, translating text, handling concurrent tasks, displaying progress bars, and creating widgets in Google Colab.

2. **Widget Creation and Display:**
   - In this part, widgets for uploading the EPUB file, selecting the target language, and a translate button are created using the `ipywidgets` library. These widgets are then displayed in the Colab notebook.

3. **Translation and Download Function:**
   - The `translate_and_download` function is defined to handle EPUB file translation and provide a download link.
   - It checks if an EPUB file is uploaded using the `upload_widget`.
   - The input EPUB file path and target language are obtained from the widgets.
   - The `translate_epub` function is called to perform the translation.
   - After translation, a download link for the translated EPUB file is created using the `base64` module to encode the binary content as base64.
   - The download link is displayed using HTML.

4. **Button Click Event Handling:**
   - The `translate_button` widget is registered with the `translate_and_download` function to execute when the button is clicked.

### **Instructions for Use**:

1. Open a Google Colab notebook.

2. Copy and paste the code into a code cell.

3. Run the code cell.

4. Widgets for uploading an EPUB file and selecting a target language will be displayed.

5. Upload an EPUB file by clicking the "Upload EPUB" button.

6. Select the target language from the dropdown menu.

7. Click the "Translate" button to perform the translation.

8. After translation, a download link for the translated EPUB file will be provided. Click the link to download the translated EPUB.

This code allows you to upload an EPUB file, translate it to the selected target language, and download the translated EPUB file from your Google Colab environment.

In [ ]:
import re
from ebooklib import epub
from deep_translator import GoogleTranslator
from concurrent.futures import ThreadPoolExecutor
from tqdm import tqdm
import time
from google.colab import files
import ipywidgets as widgets
from IPython.display import display
import base64

def translate_epub(input_epub_path, output_epub_path, target_language='en', batch_size=10):
    # Open the input EPUB file
    book = epub.read_epub(input_epub_path)

    # Initialize the Google Translate API
    translator = GoogleTranslator(source='auto', target=target_language)

    def translate_text(content):
      text_fragments = extract_text_between_tags(content)
      translated_fragments = []

      for fragment in text_fragments:
          # Check if the fragment resembles a URL (http/https)
          if re.match(r'^https?://', fragment):
              translated_fragments.append(fragment)  # Keep URLs as they are
          else:
              translated_fragment = translator.translate(fragment)
              if translated_fragment is not None:
                  translated_fragments.append(translated_fragment)
              else:
                  translated_fragments.append("")  # Replace None with an empty string

      translated_content = content
      for i in range(len(text_fragments)):
          translated_content = translated_content.replace(text_fragments[i], translated_fragments[i], 1)
      return translated_content


    # Create a ThreadPoolExecutor for concurrent translation
    with ThreadPoolExecutor() as executor:
        start_time = time.time()
        for item in tqdm(book.items, desc="Translating EPUB"):
            if isinstance(item, epub.EpubHtml):
                content = item.content.decode('utf-8')
                translated_content = executor.submit(translate_text, content).result()
                item.content = translated_content.encode('utf-8')

        end_time = time.time()

    # Save the translated EPUB to the output file
    epub.write_epub(output_epub_path, book)

    # Calculate and display the execution time
    execution_time = end_time - start_time
    print(f"Translation completed in {execution_time:.2f} seconds.")

def extract_text_between_tags(input_string):
    tag_pattern = r'<[^>]+>'
    text_fragments = re.split(tag_pattern, input_string)
    text_fragments = [fragment.strip() for fragment in text_fragments if fragment.strip()]
    return text_fragments

# Create widgets for input
languages = {
    "Afrikaans": "af",
    "Albanian": "sq",
    "Amharic": "am",
    "Arabic": "ar",
    "Basque": "eu",
    "Bengali": "bn",
    "Bosnian": "bs",
    "Bulgarian": "bg",
    "Burmese": "my",
    "Chinese": "zh",
    "Croatian": "hr",
    "Czech": "cs",
    "Danish": "da",
    "Dutch": "nl",
    "English": "en",
    "Estonian": "et",
    "Fijian": "fj",
    "Filipino": "fil",
    "Finnish": "fi",
    "French": "fr",
    "German": "de",
    "Greek": "el",
    "Gujarati": "gu",
    "Haitian Creole": "ht",
    "Hebrew": "he",
    "Hindi": "hi",
    "Hungarian": "hu",
    "Icelandic": "is",
    "Indonesian": "id",
    "Irish": "ga",
    "Italian": "it",
    "Japanese": "ja",
    "Kannada": "kn",
    "Khmer": "km",
    "Korean": "ko",
    "Kurdish": "ku",
    "Lithuanian": "lt",
    "Macedonian": "mk",
    "Malay": "ms",
    "Maltese": "mt",
    "Maori": "mi",
    "Marathi": "mr",
    "Norwegian": "no",
    "Persian": "fa",
    "Polish": "pl",
    "Portuguese": "pt",
    "Punjabi": "pa",
    "Romanian": "ro",
    "Russian": "ru",
    "Samoan": "sm",
    "Serbian": "sr",
    "Sinhala": "si",
    "Slovak": "sk",
    "Slovenian": "sl",
    "Spanish": "es",
    "Swahili": "sw",
    "Swedish": "sv",
    "Tamil": "ta",
    "Telugu": "te",
    "Thai": "th",
    "Tongan": "to",
    "Turkish": "tr",
    "Ukrainian": "uk",
    "Urdu": "ur",
    "Vietnamese": "vi",
    "Welsh": "cy",
    "Xhosa": "xh",
    "Zulu": "zu",
}

# upload_widget = widgets.FileUpload(accept='.epub', description='Upload EPUB')
uploaded = files.upload()
input_epub_path = list(uploaded.keys())[0]
language_dropdown = widgets.Dropdown(options=languages, description='Target Language:')
translate_button = widgets.Button(description='Translate')
output_link = widgets.Output()

# Display widgets
if input_epub_path.endswith('.epub' or '.EPUB'):
  display(language_dropdown, translate_button, output_link)
else:
  print('Please upload a valid EPUB file.')

# Define a function to create and display the download button
def display_download_button(output_epub_path):
    with open(output_epub_path, 'rb') as f:
        epub_blob = f.read()
        base64_epub_blob = base64.b64encode(epub_blob).decode('utf-8')
        download_link = f'<a href="data:application/epub+zip;base64,{base64_epub_blob}" download="{output_epub_path}"><button style="width: fit-content;" class="lm-Widget p-Widget jupyter-widgets jupyter-button widget-button mod-success">Download Translated EPUB</button></a>'
        display(widgets.HTML(download_link, raw=True))

# Define a function to handle EPUB translation and download
def translate_and_download(change):
    with output_link:
        output_link.clear_output()
        if not uploaded:
            print("Please upload an EPUB file.")
            return

        # Process uploaded EPUB file and display translation options
        target_language = language_dropdown.value
        output_epub_path = f'{input_epub_path.rsplit(".", 1)[0]}-tra-{target_language}.epub'

        # Translate the input EPUB file
        translate_epub(input_epub_path, output_epub_path, target_language)

        # Display a download button with styling
        display_download_button(output_epub_path)

def custom_url_encode(input_string):
    encoded_string = ""
    for char in input_string:
        if char.isalnum() or char in ['-', '_', '.', '~']:
            encoded_string += char
        else:
            encoded_string += f"%{ord(char):02X}"
    return encoded_string

# Register the translate_and_download function to run when the button is clicked
translate_button.on_click(translate_and_download)
